Connect to Google Drive:

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/MP'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/MP


Import required modules:

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
import torch

Set seed for PyTorch random number generation:

In [3]:
torch.random.manual_seed(0)

Open the original dataset and print its column:

In [4]:
df = pd.read_csv("Datasets/new_dataset.csv", encoding="utf-16", sep="\t")
print(df.columns)

<ipython-input-4-cd1e7233d4b2>:1: DtypeWarning: Columns (24,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Datasets/new_dataset.csv", encoding="utf-16", sep="\t")


Index(['id', '@context', '@type', 'claimReviewed', 'datePublished', 'url',
       'author.@type', 'author.name', 'author.url', 'itemReviewed.@type',
       'itemReviewed.author.name', 'itemReviewed.author.@type',
       'itemReviewed.author.sameAs', 'reviewRating.@type',
       'reviewRating.ratingValue', 'reviewRating.alternateName',
       'author.image', 'itemReviewed.name', 'itemReviewed.datePublished',
       'itemReviewed.firstAppearance.url', 'itemReviewed.firstAppearance.type',
       'itemReviewed.author.image', 'reviewRating.ratingExplanation',
       'itemReviewed.author.jobTitle', 'reviewRating.bestRating',
       'reviewRating.worstRating', 'reviewRating.image',
       'itemReviewed.appearance.1.url', 'itemReviewed.appearance.1.@type',
       'itemReviewed.appearance.2.url', 'itemReviewed.appearance.2.@type',
       'itemReviewed.appearance.3.url', 'itemReviewed.appearance.3.@type',
       'itemReviewed.appearance.4.url', 'itemReviewed.appearance.4.@type',
       'itemRevi

Load the model:

In [5]:
model_name = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    trust_remote_code=True,
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name, device = "cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prepare the input arguments for the LLM:

In [6]:
system_prompt = """You are an assisstant that helps fact-checking claims.
Given a claim and the label telling its veracity, you need to normalize the label to standart format,
between this choices: True, Mostly False, False, Mixture. Answer only with the label."""

a = 100
claim = df["claimReviewed"].iloc[a]
label = df["reviewRating.alternateName"].iloc[a]

claim_review = f"Claim: {claim}\nLabel: {label}"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": claim_review}
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "do_sample": False,
}

Print the results:

In [7]:
print(claim_review)
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

Claim: Benigno Aquino Jr. cofounded CPP-NPA
Label: False


The claim that Benigno Aquino Jr. cofounded the Communist Party of the Philippines-New People's Army (CPP-NPA) is indeed false. Benigno Aquino Jr., also known as Ninoy Aquino, was a prominent Filipino politician and a critic of the Marcos regime. He was assassinated in 1983, and his death became a catalyst for the People Power Revolution, which led to the end of Ferdinand
